# Value Investor Suite

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from colorama import Fore
from tabulate import tabulate
import googlesearch as gs
import requests
from bs4 import BeautifulSoup

pd.options.mode.chained_assignment = None

## Stocks file opening

In [ ]:
## STOCKS FILE OPENING

# The Excel file is open and saved into 'df_stocks'
df_stocks = pd.read_excel('stocks_list.xls')

In [ ]:
## SECTORS CHECKS

# The sectors present into the Excel file are compared with the currently known ones,
# in order to understand if new sectors have been defined or if some have been removed
sectors_list = df_stocks['Sector'].unique().tolist()
current_known_sectors = ['SERVIC', 'FINANC', 'HEALTH', 'TECHNO', 'BASICM', 
'CAPGDS','UTILIT','ENERGY','CYCLIC','NONCYC', 'TRANSP']

# Check that no new sectors are present
if set(sectors_list) != set(current_known_sectors):
    unknown_sectors = set(current_known_sectors) - set(sectors_list)
    print('-- SECTORS CHECKS --\nThe current known sectors are different from the ' +
     'ones contained into the Excel file.\n\nChange them into the list ' +
     '\'current_known_sectors\' and into the comment of the ' +
     '\'main_value_investor_suite.ipynb\'\n\n')
    print("The sectors NOT known are the following: " +
          str([str(sec) + "\n" for sec in unknown_sectors]))

In [ ]:
## MARKETS CHECKS

# The markets present into the Excel file are compared with the currently known ones,
# in order to understand if new markets have been defined or if some have been removed
markets_list = df_stocks['PV'].unique().tolist()
current_known_markets = ['EQUIDUCT', 'NASDAQ', 'EURONEXT', 'NYSE', 'XETRA', 'SIBE',
'TSX', 'LSE', 'AFF', 'VIRTX', 'AMEX', 'EURONEXTNL', 'HEX', 'EURONEXTP']

# Check that no new markets are present
if set(markets_list) != set(current_known_markets):
    print('-- MARKETS CHECKS --\nThe current known markets are different from the ' +
     'ones contained into the Excel file.\n\nChange them into the list ' +
     '\'current_known_markets\' and into the comment of the ' +
     '\'main_value_investor_suite.ipynb\'\n\n')

## Web page scraping

In [ ]:
## FUNCTION GOAL: Inspect the web page found at the url passed as input
## INPUT:  - 'url', the link where the web page of the stock cosidered can be found
##         - 'functionality', the options in which there's the interest
##         - 'stock', the name of the stock analysed
## OUTPUT: - 'years', the fiscal years of the financial numbers
##         - 'values', the financial numbers of the company analysed

def obtain_web_page_data(url, functionality, stock):      
    
    # The web page is open thanks to the 'get' command of the 'requests' library
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)' +
    ' AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    resp = requests.get(url, headers = headers)
    resp_cap = requests.get(url.split('/financials')[0], headers = headers)
    
    # The status must be 200 in order to have had a positive feedback
    if resp.status_code == 200:

        find_name = []
        keys_values = []
        to_find_s = []

        match functionality:
            case 'capitalizations':
                find_name = "div"
                keys_values = ["class", "tablesorter ratesTable"]
                to_find_s = ["tr", "td"]
            case 'financials':
                voices_list = []
                if "income-statement" in url:
                    voices_list = ['Sales/Revenue', 'Net Income', 'EPS (Basic)']
                elif "balance-sheet" in url:
                    voices_list = ['Total Shareholders\' Equity']
                elif "cash-flow" in url:
                    voices_list = ['Sales/Revenue', 'Net Income', 'EPS (Basic)']
                else:
                    print("The web page analysed is NOT a financial ")
            case '_':
                print(0)

        for i in range(0, int(len(keys_values)/2)+1):
            soup = BeautifulSoup(resp_cap.text,'html.parser')
            web_page_content = soup.find(find_name, {keys_values[i]: keys_values[i+1]})

            if web_page_content is None:
                return 0
            
            for to_find in to_find_s:
                values = []
                for i in web_page_content.findAll(to_find):
                    values.append(i.text)
    else:
        print("The stock " + stock + " cannot be found")
        values = []

    return values

In [ ]:
url = 'https://www.x-rates.com/table/?from=USD&amount=1'
functionality = 'capitalizations'
stock = ''
obtain_web_page_data(url, functionality, stock)

## Sector statistical analysis

### Sector stocks

In [ ]:
## FUNCTION GOAL: Filter the dataframe 'df_stocks' in order to find the ones which
##                satisfy the sector and market requirement
## INPUT:  - 'sector', the first condition for the filter
##         - 'market', the second condition for the filter
## OUTPUT: - 'df_stocks_filt', the dataframe filtered following the conditions

def sector_stocks_selection(sector, market):

    # The filter is done on 'sector' and, if present, also on 'market'
    if market in markets_list:
        df_stocks_filt = df_stocks[(df_stocks['Sector'] == sector) &
                                (df_stocks['PV'] == market)]
    else:
        df_stocks_filt = df_stocks[df_stocks['Sector'] == sector]

    if len(df_stocks_filt) <= 1:
        raise Exception("The stocks for sector " + str(sector) + " are less than 2")

    # The filtered and all stocks are counted to understand the percentage of stocks
    # for the current sector over all the stocks
    filtered_stocks = df_stocks_filt['Sector'].count()
    all_stocks = df_stocks['Sector'].count()

    # Print of the stats
    print("\nThe number of stocks for the column \'Sector\'" +
        " equal to the sector \'" + sector + "\' is:\n\n" + 
        str(filtered_stocks) + " over " +
        str(all_stocks) + " --> " +
        str(round(100*filtered_stocks/all_stocks, 2)) +
        " % of the total\n\n" +
        "_____________________________________________________\n\n")
    tabulate(df_stocks_filt.head(), headers='keys', tablefmt='psql')

    return df_stocks_filt

### P/E Ratios stats plots

In [ ]:
## FUNCTION GOAL: Analyse the P/E ratios of the stocks filtered obtained as output
##                of the function 'sector_stocks_selection()' and define the relevant
##                percentiles
## INPUT:  - 'df_stocks_filt', output of the function 'sector_stocks_selection()'
##         - 'stats_tf', True or False if the statistics infos must be displayed
##         - 'plot_tf', True or False if the plots must be displayed
## OUTPUT: - 'pe_perctile30', the 30-th percentile of the P/E ratios
##         - 'df_stocks_filt', the same dataframe, without the stocks which have
##           'nan' P/E ratios

def pe_ratios_stats_definition(df_stocks_filt, sector, stats_tf = False, plot_tf = False):
    
    # The stocks which do not have a P/E (so the ones which are losing money)
    # are not considered
    df_stocks_filt = df_stocks_filt[pd.to_numeric(df_stocks_filt['PERatio'],
                                                  errors='coerce').notnull()]
    df_stocks_filt.PERatio = pd.to_numeric(df_stocks_filt.PERatio)
    pe_ratios = list(df_stocks_filt.PERatio)

    # The relevant stats are saved
    stats = [np.mean(pe_ratios), np.std(pe_ratios), np.percentile(pe_ratios, 5),
            np.percentile(pe_ratios, 30), np.percentile(pe_ratios, 70), 
            np.percentile(pe_ratios, 95)]

    # The 30-th percentile is memorized
    pe_perctile30 = stats[3]

    # Print stats
    if stats_tf == True:
        print('Price/Earning Ratio Stats\n\n \tAverage: ' + str(round(stats[0],2)) +
            '\n\tStandard deviation: ' + str(round(stats[1],2)) +
            '\n\t5th percentile: ' + str(round(stats[2], 2)) +
            '\n\t30th percentile: ' + str(round(stats[3], 2)) +
            '\n\t70th percentile: ' + str(round(stats[4], 2)) +
            '\n\t95th percentile: ' + str(round(stats[5], 2)))

    # Plot display
    if plot_tf == True:
        # Only the P/E under the 95-th percentile are kept, to avoid outliers
        pe_ratios_adj = [value for value in pe_ratios if value <= stats[5]]

        # P/E density display
        plt.figure(figsize=(14,7))
        plt.style.use("seaborn-v0_8") #plt.style.use('seaborn-whitegrid')
        plt.hist(pe_ratios_adj, bins=30,
            density=True, facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)
        plt.axvline(stats[2], color='b', linestyle='dashed',
            linewidth=2.5, label='5-th percentile')
        plt.axvline(stats[3], color='g', linestyle='dashed',
            linewidth=2.5, label='30-th percentile')
        plt.axvline(stats[0], color='k', linestyle='dashed', 
            linewidth=2.5, label='Mean')
        ticks = [round(elem, 1) for elem in [stats[0],stats[2],stats[3]]]
        plt.xticks(ticks, ticks, fontsize=14)
        font = {'family': 'serif',
                #'color':  'darkred',
                'weight': 'normal',
                'size': 18,
                }
        plt.title('P/E Ratios Distribution - ' + 'Sector' + ': ' + sector, 
            fontsize=20, fontdict=font) 
        plt.xlabel('P/E', fontsize=18, fontdict=font) 
        plt.ylabel('Density', fontsize=18, fontdict=font)
        plt.legend(loc=0, prop=font)
        plt.show()

        # Boxplot display
        plt.figure(figsize=(10,5))
        plt.title('Boxplot', fontsize=20, fontdict=font)
        plt.xlabel('P/E', fontsize=18)
        sns.boxplot(pe_ratios_adj, orient = 'h', fliersize = 1, linewidth = 2)
        plt.show()

        print("_____________________________________________________\n\n")

    return df_stocks_filt, pe_perctile30

### Capitalization stats plots

In [ ]:
## FUNCTION GOAL: Analyse the capitalizations of the stocks filtered obtained as output
##                of the function 'sector_stocks_selection()' and define the relevant
##                percentiles
## INPUT:  - 'df_stocks_filt', output of 'sector_stocks_selection' function
##         - 'stats_tf', True or False if the statistics infos must be displayed
##         - 'plot_tf', True or False if the plots must be displayed
## OUTPUT: - 'pe_perctile30', the 30-th percentile of the P/E ratios
##         - 'df_stocks_filt', the same dataframe, without the stocks which have
##           'nan' capitalizations

def capitalization_stats_definition(df_stocks_filt, sector, stats_tf, plot_tf):

    # The stocks which do not have a capitalization are not considered
    df_stocks_filt = df_stocks_filt[pd.to_numeric(df_stocks_filt['Capitalization'],
                                                  errors='coerce').notnull()]

    # All the capitalizations are converted to dollars, in order to have
    # the same currency for all the stocks
    url_changes = 'https://www.x-rates.com/table/?from=USD&amount=1'
    changes = obtain_web_page_data(url_changes, 'capitalizations', '')

    df_stocks_filt.Capitalization = pd.to_numeric(df_stocks_filt.Capitalization)
    capitalizations = list(df_stocks_filt.Capitalization)

    # The relevant stats are saved
    stats = [np.mean(capitalizations), np.std(capitalizations),
            np.percentile(capitalizations, 5), np.percentile(capitalizations, 30),
            np.percentile(capitalizations, 70), np.percentile(capitalizations, 95)]

    # The 70-th percentile is memorized
    cap_perctile70 = stats[4]

    # Print stats
    if stats_tf == True:
        print('Capitalization Stats\n\n \tAverage: ' + str(round(stats[0],2)) + ' $' +
            '\n\tStandard deviation: ' + str(round(stats[1],2)) +
            '\n\t5th percentile: ' + str(round(stats[2], 2)) + ' $' +
            '\n\t30th percentile: ' + str(round(stats[3], 2)) + ' $' +
            '\n\t70th percentile: ' + str(round(stats[4], 2)) + ' $' +
            '\n\t95th percentile: ' + str(round(stats[5], 2)) + ' $')

    # Plot display
    if plot_tf == True:
        # Only the capitalizations under the 95-th percentile are kept, to avoid outliers
        capitalizations_adj = [value for value in capitalizations if value <= stats[5]]

        # Capitalizations density display
        plt.figure(figsize=(14,7))
        plt.style.use("seaborn-v0_8") #plt.style.use('seaborn-whitegrid')
        plt.hist(capitalizations_adj, bins=30,
            density=True, facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)
        plt.axvline(stats[2], color='b', linestyle='dashed',
            linewidth=2.5, label='5-th percentile')
        plt.axvline(stats[4], color='g', linestyle='dashed',
            linewidth=2.5, label='70-th percentile')
        plt.axvline(stats[0], color='k', linestyle='dashed', 
            linewidth=2.5, label='Mean')
        ticks = [round(elem, 1) for elem in [stats[0],stats[2],stats[4]]]
        plt.xticks(ticks, ticks, fontsize=14)
        font = {'family': 'serif',
                #'color':  'darkred',
                'weight': 'normal',
                'size': 18,
                }
        plt.title('Capitalizations Distribution - ' + 'Sector' + ': ' + sector, 
            fontsize=20, fontdict=font) 
        plt.xlabel('Capitalization', fontsize=18, fontdict=font) 
        plt.ylabel('Density', fontsize=18, fontdict=font)
        plt.legend(loc=0, prop=font)
        plt.show()

        # Boxplot display
        plt.figure(figsize=(10,5))
        plt.title('Boxplot', fontsize=20, fontdict=font)
        plt.xlabel('Capitalization', fontsize=18)
        sns.boxplot(capitalizations_adj, orient = 'h', fliersize = 1, linewidth = 2)
        plt.show()

        print("_____________________________________________________\n\n")

    return df_stocks_filt, cap_perctile70

## Selected stocks analysis

### Best sector stocks

In [ ]:
# Metrics to be implemented:
# (1) P/E 5 years < 25
# (2) P/E 12 months < 20
# (3) Capitalization > 10.000.000.000
# (4) Capitalization > p_70
# (5) P_stock > 0.5*Capitalization/Stock_no
# (6) P_stock < 1.33*TAV/Stock_no (TAV = physical properties company)
# (7) Dividends years payment > 10
# (8) Revenue_i > Revenue_(i-1)
# (9) (OE_i - OE_(i-1))/OE_(i-1) > 7%
# (10) Year income > 2.000.000.000
# (11) Current activities > 2*Current debts
# (12) P/B < 1.5

In [ ]:
## FUNCTION GOAL: Find the best stocks for the sector and for the market defined
## INPUT:  - 'df_stocks_filt', output of 'sector_stocks_selection' function
##         - 'pe_perctile30', the 30-th percentile of the P/E ratios
##         - 'cap_perctile70', the 70-th percentile of the capitalizations
##         - 'sector', the one chosen as filter
##         - 'market', the one chosen as filter
## OUTPUT: - 'df_stocks_filt_final', the dataframe with the best stocks of the sector
##           and market which satisfy the conditons needed
##         - 'stocks_to_search', the list with the stock names

def best_stocks_selection(df_stocks_filt, pe_perctile30, cap_perctile70, sector, market):

    # The filter is done considering the stocks which:
    # (1) Have a P/E ratio < 15
    # (2) Have a P/E ratio < pe_perctile30
    # (3) Have a capitalization > 10.000.000.000
    # (4) Have a capitalization > cap_perctile70
    # (5) Distribute the dividend
    df_stocks_filt_final = df_stocks_filt[(df_stocks_filt.PERatio 
                                <= min(pe_perctile30, 15)) &
                                (df_stocks_filt.Capitalization
                                >= max(cap_perctile70, 1e4))]
    df_stocks_filt_final = df_stocks_filt_final[pd.to_numeric(
                                        df_stocks_filt_final['DividendYield'],
                                            errors='coerce').notnull()].sort_values(
                                                by='Capitalization', ascending = False)

    # Printing of the first line of the display
    if market in markets_list:
        print(Fore.GREEN + "\nBest stocks for the sector \'" + sector +
            "\' in the market \'" + market + "\'\n")
    else:
        print(Fore.GREEN + "\nBest stocks for the sector \'" + sector + "\'\n")

    # Only the names of the selected stocks is saved into a list to be used later
    stocks_to_search = df_stocks_filt_final['Description'].to_list()

    # Display of the dataframe found
    print(Fore.WHITE + 
        tabulate(df_stocks_filt_final[['InstrID', 'PV', 'Description', 'PERatio',
                    'DividendYield', 'Capitalization']], headers='keys',
                    tablefmt='psql', showindex=False))

    print("_____________________________________________________\n\n")

    return df_stocks_filt_final, stocks_to_search

### Financial analysis

In [ ]:
def financial_analysis(stocks_to_search, analysis_type):

    for stock in stocks_to_search:
        print("\n\nThe stock currently analysed is: " + str(stock) + "\n")
        #query = "lt.morningstar.com " + stock
        query = stock + " investing financials marketwatch" #wsj"
        print(query)
    
        url = gs.lucky(query, num=1)

        print(url)

        # Summary analysis
        url_sum = (url.split('/financials')[0] + url.split('/financials')[1])
        [headers, voices_list, values] = obtain_web_page_data(url_sum, stock)

        # Income statement analysis
        [headers, voices_list, values] = obtain_web_page_data(url, stock)

        # Balance sheet analysis
        url_bs = (url.split('/financials')[0] + '/financials/balance-sheet' + 
               url.split('/financials')[1])
        [headers, voices_list, values] = obtain_web_page_data(url_bs, stock)

        # Cash flow analysis
        url_cf = (url.split('/financials')[0] + '/financials/cash-flow' + 
               url.split('/financials')[1])
        [headers, voices_list, values] = obtain_web_page_data(url_cf, stock)
        

        headers
        for voice in voices_list:
            idx = values.index(voice)
            values[idx+1:idx+6]
        
        #print(headers)
        #print(values)

    return 0

## Value investor suite function

In [ ]:
def main(sector, market, stats_tf, plot_tf):

    df_stocks_filt = sector_stocks_selection(sector, market)

    [df_stocks_filt, pe_perctile30] = pe_ratios_stats_definition(
        df_stocks_filt, sector, stats_tf, plot_tf)

    [df_stocks_filt, cap_perctile70] = capitalization_stats_definition(
        df_stocks_filt, sector, stats_tf, plot_tf)

    [df_stocks_filt_final, stocks_to_search] = best_stocks_selection(
        df_stocks_filt, pe_perctile30, cap_perctile70, sector, market)

    return 0

In [ ]:
# TODO:
# (1) Complete the morningstar search by memorizing all the information
# (2) Analyse all the conditions needed for a good performance
# (3) Return a grade for every filtered stock
# (4) Calculate P/E over the last 5 years